In [1]:
%%bash

cp ../src/isanlp_rst/annotation.py ~/.pyenv/versions/3.6.7/lib/python3.6/site-packages/isanlp/annotation_rst.py
cp ../src/isanlp_rst/annotation_rst_pb2.py ~/.pyenv/versions/3.6.7/lib/python3.6/site-packages/isanlp/annotation_rst_pb2.py
cp ../src/isanlp_rst/annotation_from_protobuf.py ~/.pyenv/versions/3.6.7/lib/python3.6/site-packages/isanlp/annotation_from_protobuf.py

In [2]:
from isanlp.processor_remote import ProcessorRemote
from isanlp.processor_syntaxnet_remote import ProcessorSyntaxNetRemote
from isanlp import PipelineCommon
from isanlp.ru.converter_mystem_to_ud import ConverterMystemToUd

address_morph = (SERVER0, 4333)
address_syntax = (SERVER0, 4340)
address_rst = (SERVER1, 3490)

ppl = PipelineCommon([(ProcessorRemote(address_morph[0], address_morph[1], 'default'),
                 ['text'],
                 {'tokens': 'tokens',
                  'sentences': 'sentences',
                  'postag': 'mystem_postag',
                  'lemma': 'lemma'}),
                (ProcessorSyntaxNetRemote(address_syntax[0], address_syntax[1]),
                 ['tokens', 'sentences'],
                 {'syntax_dep_tree': 'syntax_dep_tree'}),
                (ConverterMystemToUd(),
                 ['mystem_postag'],
                 {'morph': 'morph',
                  'postag': 'postag'}),
                (ProcessorRemote(address_rst[0], address_rst[1], 'default'),
                 ['text', 'tokens', 'sentences', 'postag', 'morph', 'lemma', 'syntax_dep_tree'],
                 {'rst': 'rst'})])

In [3]:
res = ppl('Тем не менее, по данным РИА Новости, инициатива была одобрена комитетом.')

In [4]:
print(res['rst'])

In [5]:
def extr_pairs(tree):
    pp = []
    
    if tree.left:
        pp.append([tree.left.text, tree.right.text, tree.relation])
        pp += extr_pairs(tree.left)
        pp += extr_pairs(tree.right)
    return pp

In [6]:
print(res['rst'][0])

id: 14
text: Тем не менее, по данным РИА Новости, инициатива была одобрена комитетом.
relation: root
left: Тем не менее, по данным РИА Новости, инициатива была
ight:  одобрена комитетом.



In [7]:
extr_pairs(res['rst'][0])

[['Тем не менее, по данным РИА Новости, инициатива была',
  ' одобрена комитетом.',
  'root'],
 ['Тем не', ' менее, по данным РИА Новости, инициатива была', 'relation'],
 [' менее, по данным РИА Новости', ', инициатива была', 'relation'],
 [' менее', ', по данным РИА Новости', 'relation'],
 [', инициатива', ' была', 'relation'],
 [' одобрена', ' комитетом.', 'relation'],
 [' комитетом', '.', 'relation']]